In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

In this notebook i implement an experimental model class that seeks to replicate the pipeline i'd use for the test set to get a more accurate validation and not just the "branches" of the schematic below.

                                          X_test
                   |                        |
                   v                        /\
                                         _FR  _DE
                                          |    |
                                      pred_FR pred_DE
                                          |    |
                                           \  /  <--------- This class shows the effect
                                           Y_pred            of this step on the scores

In [5]:
from mord import OrdinalRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


model=Model(LinearRegression,LinearRegression)
model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=100,custom_group=2,group=True)
model.Compute(iterations=100,custom_group=2,group=True,rank_before=False)
model.Compute(iterations=100,custom_group=2,group=True,split_countries=True)

model=Model(OrdinalRidge,OrdinalRidge)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=100,columns='Rank',custom_group=2,group=True)

columns are  ['TARGET', 'Rank', 'Rank_group']
average TARGET evaluation score at: 24.1    training at: 32.3     FR model at: 10.9     DE model at: 34.6
average Rank evaluation score at: 17.5    training at: 24.5     FR model at: 17.2     DE model at: 36.8
average Rank_group evaluation score at: 26.6    training at: 34.4     FR model at: 17.0     DE model at: 36.5


columns are  ['TARGET', 'Rank', 'Rank_group']
average TARGET evaluation score at: 23.9    training at: 32.0     FR model at: 10.2     DE model at: 35.2
average Rank evaluation score at: 28.4    training at: 35.4     FR model at: 16.7     DE model at: 38.4
average Rank_group evaluation score at: 27.7    training at: 34.3     FR model at: 17.1     DE model at: 37.8


columns are  ['Rank', 'Rank_group']
average Rank evaluation score at: 27.1    training at: 35.9     FR model at: 16.4     DE model at: 36.5
average Rank_group evaluation score at: 20.5    training at: 28.0     FR model at: 13.0     DE model at: 29.5




Our best bet is still a linear regression of Rank as we saw in the previous notebook it would seem ordinal regression struggles when we reunite the predicted datasets. Ranking after the split is also more devastating in this instance since we already have low scores...


In [5]:
model=Model(RandomForestRegressor,RandomForestRegressor,2)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=50,group=True,rank_before=True)

columns are  ['TARGET', 'Rank', 'Rank_group']
average TARGET evaluation score at: 11.1    training at: 72.8     FR model at: 6.2     DE model at: 15.7
average Rank evaluation score at: 12.2    training at: 77.1     FR model at: 6.5     DE model at: 17.9
average Rank_group evaluation score at: 10.1    training at: 67.6     FR model at: 4.9     DE model at: 16.0




above is just a representation of low the generalization capacity of a random forest is for this dataset

In [7]:
X_test = pd.read_csv(datasets+'X_test_final.csv')

In [14]:
print(np.unique(X_test['COUNTRY'],return_counts=True)[1]/len(X_test))
print(np.unique(X_train['COUNTRY'],return_counts=True)[1]/len(X_train))

[0.44189602 0.55810398]
[0.43038822 0.56961178]


This is a test to study the structures of the datasets which led to the implementation of split_country argument in the Compute method of the model in case Sklearn split_train_test() function selected mostly rows with FR and not much DE for example. (To remedy to this we spit countries and then determine the test data stets by country and concatenate them to have full test set) but it does not seem to affect performance much. as we see in the linear regression cells. But it is a better method to evaluate i believe

Pretty similar structure. split into countries then split into test/train guarantees similar distribution
or split into test/train then split 